In [1]:
import os
os.chdir('/workspaces/work_utils2')

In [2]:
import presentation

In [3]:
ppt = presentation.PowerPointPresentation(template_filepath='presentation/assets/Blank.pptx')


In [5]:
slide = ppt.add_slide('Title Slide')